In [1]:
# print cwd
import os
os.getcwd()

'c:\\Users\\cayde\\Desktop\\Grad_School_stuff\\DataBaseManagement\\Project\\Analysis_Scripts\\mycode'

In [2]:
# store path to data directory
data_dir = os.path.join(os.getcwd(), '../data/test') # '../data/CSMiningData'  '../data/test'
print(data_dir)

# store the number of files in the data directory
num_files = len([name for name in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, name))])
print(num_files)

c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test
3


In [4]:
import os
import numpy as np
from scipy.stats import entropy
from collections import Counter

def calc_kld_jsd(file1, file2):
    """
    Calculates Kullback-Leibler divergence and Jensen-Shannon Divergence for two files.

    Args:
    file1 (str): path to the first file.
    file2 (str): path to the second file.

    Returns:
    tuple: Kullback-Leibler divergence of file1 from file2, Kullback-Leibler divergence of file2 from file1, and Jensen-Shannon Divergence between file1 and file2.
    """

    # read files
    try:
        with open(file1, "r") as inpFile1:
            # split file into words
            words1 = inpFile1.read().split()
    except FileNotFoundError:
        print(f"Error: File {file1} not found.")
        return None

    try:
        with open(file2, "r") as inpFile2:
            words2 = inpFile2.read().split()
    except FileNotFoundError:
        print(f"Error: File {file2} not found.")
        return None

    if not words1 or not words2:
        print(f"Error: One or both input files are empty.")
        return None

    try:
        # count frequency of words in each file. This will be the probability distribution of each file.
        word_freq1 = Counter(words1)
        word_freq2 = Counter(words2)

        # get unique words from both files. This will be the set of all words in both files. 
        unique_words = set(word_freq1.keys()).union(set(word_freq2.keys()))

        # calculate probability distribution for each file. If a word is not present in a file, its probability is 0. 
        counts1 = np.array([word_freq1.get(word, 0) for word in unique_words], dtype=np.float64)
        counts2 = np.array([word_freq2.get(word, 0) for word in unique_words], dtype=np.float64)

        # normalize the probability distributions to sum to 1.
        file1_probs = counts1 / counts1.sum()
        file2_probs = counts2 / counts2.sum()

        # calculate Kullback-Leibler divergence and Jensen-Shannon Divergence.
        # KLD is a directed measure of the difference between two probability distributions.
        KL1 = entropy(file1_probs, file2_probs)
        KL2 = entropy(file2_probs, file1_probs)

        # JSD is a symmetrized and smoothed version of KLD. It is a measure of similarity between two probability distributions. 
        M = 0.5 * (file1_probs + file2_probs)
        JSD = 0.5 * (entropy(file1_probs, M) + entropy(file2_probs, M))

        return KL1, KL2, JSD

    except ValueError:
        print(f"Error: Failed to calculate KLD and JSD for files {file1} and {file2}.")
        return None

def compare_all_files(data_dir):
    """
    Compares all files in a directory using Kullback-Leibler divergence and Jensen-Shannon Divergence.

    Args:
    data_dir (str): path to the directory containing the files.

    Returns:
    tuple: Matrix of Kullback-Leibler divergences and Matrix of Jensen-Shannon Divergences.
    """

    # get list of all files in the directory
    file_names = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]

    # get number of files
    num_files = len(file_names)

    # initialize matrices to store KLD and JSD values
    KLDMatrix = np.zeros((num_files, num_files))
    JSDMatrix = np.zeros((num_files, num_files))

    # calculate KLD and JSD for all pairs of files
    for i in range(num_files):
        for j in range(i+1, num_files):
            # calculate KLD and JSD for files i and j
            KL1, KL2, JSD = calc_kld_jsd(file_names[i], file_names[j])

            # store KLD and JSD values in the matrices
            KLDMatrix[i, j] = KL1
            KLDMatrix[j, i] = KL2
            JSDMatrix[i, j] = JSD
            JSDMatrix[j, i] = JSD

            print(f"Comparison complete for files: {file_names[i]} and {file_names[j]}")

    return KLDMatrix, JSDMatrix

KLDMatrix, JSDMatrix = compare_all_files(data_dir)

try:
    np.savetxt("output/KLDMatrix.csv", KLDMatrix, delimiter=",")
    np.savetxt("output/JSDMatrix.csv", JSDMatrix, delimiter=",")
except IOError:
    print("Error: Failed to write output to file.")


Comparison complete for files: c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace1.txt and c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace2.txt
Comparison complete for files: c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace1.txt and c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace3.txt
Comparison complete for files: c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace2.txt and c:\Users\cayde\Desktop\Grad_School_stuff\DataBaseManagement\Project\Analysis_Scripts\mycode\../data/test\Trace3.txt


In [14]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

# read in the KLD and JSD matrices
KLDMatrix = np.genfromtxt("output/CSMiningData/KLDMatrix.csv", delimiter=",")
JSDMatrix = np.genfromtxt("output/CSMiningData/JSDMatrix.csv", delimiter=",")

# Prim's Algorithm
# Prim's algorithm constructs the minimum spanning tree by adding edges with the minimum weight at each step.

# JSDMatrix
matrix_sparse = csr_matrix(JSDMatrix)
Tcsr = minimum_spanning_tree(matrix_sparse)

# save the minimum spanning tree as a csv file
try:
    np.savetxt("output/CSMiningData/prim_mst_JSD.csv", Tcsr.toarray(), delimiter=",")
except IOError:
    print("Error: Failed to write output to file.")

# KLDMatrix
matrix_sparse = csr_matrix(KLDMatrix)
Tcsr = minimum_spanning_tree(matrix_sparse)

# save the minimum spanning tree as a csv file
try:
    np.savetxt("output/CSMiningData/prim_mst_KLD.csv", Tcsr.toarray(), delimiter=",")
except IOError:
    print("Error: Failed to write output to file.")

In [13]:
import numpy as np
import networkx as nx

# read in the KLD and JSD matrices
KLDMatrix = np.genfromtxt("output/CSMiningData/KLDMatrix.csv", delimiter=",")
JSDMatrix = np.genfromtxt("output/CSMiningData/JSDMatrix.csv", delimiter=",")

# Kruskal's Algorithm
# If you decide to implement Kruskal's algorithm, the pseudocode is as follows:

# Create a graph from your matrices
G_KLD = nx.from_numpy_matrix(KLDMatrix)
G_JSD = nx.from_numpy_matrix(JSDMatrix)

# Compute the minimum spanning tree for both matrices
MST_KLD = nx.minimum_spanning_tree(G_KLD)
MST_JSD = nx.minimum_spanning_tree(G_JSD)

# Convert back to numpy matrices
MST_matrix_KLD = nx.to_numpy_matrix(MST_KLD)
MST_matrix_JSD = nx.to_numpy_matrix(MST_JSD)

# save the minimum spanning trees as csv files
try:
    np.savetxt("output/CSMiningData/kruskals_mst_KLD.csv", MST_matrix_KLD, delimiter=",")
    np.savetxt("output/CSMiningData/kruskals_mst_JSD.csv", MST_matrix_JSD, delimiter=",")
except IOError:
    print("Error: Failed to write output to file.")
